In [1]:
import os, re
import numpy as np
import pandas as pd
import keras
import keras_bert
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
# 請從google storage下載news_cheat_model_ver_01.h5
model = keras.models.load_model( "/xxxxx/news_cheat_model_ver_04.h5",
                                 custom_objects = keras_bert.get_custom_objects()
                               )

/opt/virtualenvs/r-tensorflow/lib/python3.5/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
folderPath      = "/vocab放置Folder的位置/"
dict_path       = os.path.join( folderPath , "vocab.txt"  )

In [5]:
token_dict = keras_bert.load_vocabulary(dict_path)
tokenizer  = keras_bert.Tokenizer(token_dict)

In [18]:
# 底下為詐騙文章，詐騙者:陳瑞芳
#text   = "施柏成,新聞標題：南檢查獲走私中國未檢疫魚貨5千餘公斤 船長貨主等7人收押。文章內容:台南地檢署查獲振瑞發88號漁船涉走私未經檢疫的中國魚貨5000餘公斤進入台南安平港。（記者王俊忠翻攝）〔記者王俊忠／台南報導〕台南地檢署偵辦振瑞發88號漁船涉於今年1月21日走私未經檢疫的中國魚貨5000餘公斤進入台灣安平港，經檢察官將貨主陳澤信、船長陳瑞芳及5名外籍漁工向法院聲請羈押均獲准；潘姓輪機長經檢察官諭知交保2萬元。台南地檢署承辦檢察官王宇承於2018年8月間獲報有漁船走私魚貨情資，即指揮海巡署台南機動查緝隊、嘉義查緝隊、南市刑大、第六警分局成立專案小組，歷經5個多月蒐證，於今年1月21日查獲陳澤信、陳瑞芳及5名中國、印尼籍漁工陳某、順某等駕駛漁船到中國地區領海，以海上接駁方式將中國地區未經檢疫的魚產品紅魽4793公斤、春子魚348公斤，佯裝成自行撈捕的魚貨，從台南安平港入關。檢警專案小組查出此漁船走私約5141公斤的中國地區魚產品入境、當場扣押。經檢察官訊問後，認為陳澤信等人涉犯懲治走私條例、入出國及移民法犯罪嫌疑重大，貨主陳澤信、船長陳瑞芳等人有串證之虞，陳姓等漁工無一定住居所，有逃亡之虞，經向法院聲請羈押均獲准。潘姓輪機長無羈押必要，檢察官諭知交保2萬元，並會深入偵辦此案。南檢襄閱主任檢察官柯怡伶表示，近期非洲豬瘟疫情於境外蔓延，全球聞之色變，且農曆春節將至，年貨、食品需求量大增，價格高漲，南檢對於未經檢疫的走私物品，均嚴加查緝、徹底究辦。籲請民眾返國勿私下夾帶肉製品或走私未經檢疫的農、魚、畜產品入境，以維護國內農、漁、畜牧業及民眾健康。台南地檢署查獲振瑞發88號漁船涉走私未經檢疫的中國魚貨5000餘公斤進入台南安平港。（記者王俊忠翻攝）"
text   = "施柏成,新聞標題：無。文章內容：Maryan Lim詐騙榮民，遭判刑6年十個月"
tokens = tokenizer.tokenize(text)
x1, x2 = tokenizer.encode(first=text , max_len=500)
x1 = np.array([x1])
x2 = np.array([x2])

In [49]:
ps1, ps2 = model.predict( [x1,x2] )
startVt , endVt = ps1[0,:,0], ps2[0,:,0]
startVt_ , endVt_ = np.where(startVt > 0.5)[0] , np.where(endVt > 0.4)[0] 
predRole = []

if len(startVt_)==0:
    # 機率值沒有超過Threshold的話，則取最大作代表(最多一個)
    startVt_ = startVt.argmax()
    end_     = endVt[startVt_:len(endVt)].argmax() + startVt_
    predRole.append("".join( token[startVt_:(end_+1)] ))
else:    
    for i in startVt_:
        j =  endVt_[ endVt_ >= i ]
        if len(j) > 0:
            j = j[0]
            token = tokenizer.tokenize(text)
            #predRole.append("".join( token[i:(j+1)] ))
            
            # 名字全是英文
            EngToken = []
            if( len( re.findall(u'[^a-z#]', "".join( token[i:(j+1)] ) ) )==0 ):
              EngToken = []
              for tokenUnit in token[i:(j+1)] :
                if re.findall(u'##',tokenUnit):
                   EngToken[len(EngToken)-1] = EngToken[len(EngToken)-1]+tokenUnit.replace("##","")   
                else:
                   EngToken.append( tokenUnit )
              predRole.append(" ".join( EngToken ))
            else:
              predRole.append("".join( token[i:(j+1)] ))

            # 名字全是中文字
            if( len( re.findall(u'[\u4e00-\u9FFF]', "".join( token[i:(j+1)] ) ) )==(j-i+1) ):
                # 名字全中文且人名超過5個字元就放棄不取，這裡有待 Enhance
                if( j - i > 5 ):
                    next              
                
            if(i==1): # 如果最大機率是取到施柏成的"施"位置，就不會再取第二個人名
                next
        
np.unique(predRole)    

['陳', '瑞', '芳']

In [50]:
# 底下非詐騙文章，Response:施柏成
text = "施柏成,新聞標題：。文章內容:冠德公司當日重大訊息之詳細內容本資料由　(上市公司)冠德　公司提供序號 1發言日期 108/12/02發言時間 17:01:55發言人 張勝安發言人職稱 副總經理發言人電話 (02)2378-6789主旨 代重要子公司環球購物中心公告會計主管及財務主管異動符合條款第 8款\t事實發生日 108/12/01說明1.人員變動別（請輸入發言人、代理發言人、重要營運主管(如:執行長、營運長、行銷長及策略長等)、財務主管、會計主管、公司治理主管、研發主管、內部稽核主管或訴訟及非訟代理人）:會計主管及財務主管2.發生變動日期:108/12/013.舊任者姓名、級職及簡歷:陳依華／財務室資深協理4.新任者姓名、級職及簡歷:會計主管:戴嘉伶／財務部經理。財務主管:范姜伯真／總經理室資深經理5.異動情形（請輸入「辭職」、「職務調整」、「資遣」、「退休」、「死亡」、「新任」或「解任」）:職務調整6.異動原因:職務調整7.生效日期:108/12/018.其他應敘明事項:無(1)繼任人選擬於最近一次董事會通過。(2)集團內部職務調整調任至冠德建設股份有限公司。以上資料均由各公司依發言當時所屬市場別之規定申報後，由本系統對外公佈，資料如有虛偽不實，均由該公司負責。"
tokens = tokenizer.tokenize(text)
x1, x2 = tokenizer.encode(first=text , max_len=400)
x1 = np.array([x1])
x2 = np.array([x2])

In [51]:
ps1, ps2 = model.predict( [x1,x2] )
startVt , endVt = ps1[0,:,0], ps2[0,:,0]
startVt_ , endVt_ = np.where(startVt > 0.5)[0] , np.where(endVt > 0.4)[0] 
predRole = []

if len(startVt_)==0:
    # 機率值沒有超過Threshold的話，則取最大作代表(最多一個)
    startVt_ = startVt.argmax()
    end_     = endVt[startVt_:len(endVt)].argmax() + startVt_
    predRole.append("".join( token[startVt_:(end_+1)] ))
else:    
    for i in startVt_:
        j =  endVt_[ endVt_ >= i ]
        if len(j) > 0:
            j = j[0]
            token = tokenizer.tokenize(text)
            #predRole.append("".join( token[i:(j+1)] ))
            
            # 名字全是英文
            EngToken = []
            if( len( re.findall(u'[^a-z#]', "".join( token[i:(j+1)] ) ) )==0 ):
              EngToken = []
              for tokenUnit in token[i:(j+1)] :
                if re.findall(u'##',tokenUnit):
                   EngToken[len(EngToken)-1] = EngToken[len(EngToken)-1]+tokenUnit.replace("##","")   
                else:
                   EngToken.append( tokenUnit )
              predRole.append(" ".join( EngToken ))
            else:
              predRole.append("".join( token[i:(j+1)] ))

            # 名字全是中文字
            if( len( re.findall(u'[\u4e00-\u9FFF]', "".join( token[i:(j+1)] ) ) )==(j-i+1) ):
                # 名字全中文且人名超過5個字元就放棄不取，這裡有待 Enhance
                if( j - i > 5 ):
                    next              
                
            if(i==1): # 如果最大機率是取到施柏成的"施"位置，就不會再取第二個人名
                next
        
np.unique(predRole)    

['施', '柏', '成']